<a href="https://jupyterhub.user.eopf.eodc.eu/hub/user-redirect/git-pull?repo=https://github.com/eopf-toolkit/eopf-101&branch=main&urlpath=lab/tree/eopf-101/22_zarr_struct_S2L2A.ipynb" target="_blank">
  <button style="background-color:#0072ce; color:white; padding:0.6em 1.2em; font-size:1rem; border:none; border-radius:6px; margin-top:1em;">
    🚀 Launch this notebook in JupyterLab
  </button>
</a>

### Introduction
This tutorial introduces you to the structure of an **EOPF Zarr** product sample for **Sentinel-2 L2A** data. We will demonstrate how to access and open a `.zarr` product sample with `xarray`, how to visualise the `zarr` encoding structure, explore embedded information, and retrieve relevant metadata for further processing.

### What we will learn

- ⚙️ How to open a `.zarr` file using `xarray`?
- 🛰️ The general structure of a Sentinel-2 L-2A item
- 🔎 How to access metadata that describes the `.zarr` encoding?

### Prerequisites
This tutorial uses a re-processed sample dataset from the [EOPF Sentinel Zarr Samples Service STAC API](https://stac.browser.user.eopf.eodc.eu/) that is available for direct access [here](https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202506-s02msil2a/10/products/cpm_v256/S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr). 

The selected `zarr` product is a Sentinel-2 L2A tile from the 10th of June 2025:

* File name: `S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr.`).

<hr>

#### Import libraries

In [11]:
import os
import xarray as xr

#### Helper functions

##### `print_gen_structure`
This function helps us to retrieve and visualise the names for each of the stored groups inside a `.zarr` product. As an output, it will print a general overview of elements inside the `zarr`.

In [12]:
def print_gen_structure(node, indent=""):
    print(f"{indent}{node.name}")     #allows us access each node
    for child_name, child_node in node.children.items(): #loops inside the selected nodes to extract naming
        print_gen_structure(child_node, indent + "  ") # prints the name of the selected nodes

<hr>

## Open a Zarr Store

In a first step, we use the `open_datatree()` function from the `xarray` library to open a `.zarr` store as a `DataTree`.<br>
Inside, we need to define the following key word arguments:<br>

- `filename_or_obj`: path leading to a `.zarr` store
- `engine`: `'eopf-zarr'`, designed for the EOPF `zarr` by ESA. 
- `op_mode`: extension by the `xarray-eopf` development for allowing an **analysis** or a **native** mode. For more information visit the [xarray-eopf](https://eopf-sample-service.github.io/xarray-eopf/) documentation.
- `chunks`: loads the data with `Dask` using the engine’s preferred chunk size, generally identical to the format’s chunk size

The final print of the `DataTree` object is commented out, as the display can be quite extensive, showing the entire content within the `.zarr`. An alternative is to apply a helper function that only displays the higher-level structure as shown in the next code cell.

In [13]:
url = 'https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202506-s02msil2a/10/products/cpm_v256/S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr'
# s2l2a_zarr_sample= xr.open_datatree(url,
#     engine="eopf-zarr", # storage format
#     op_mode="native", # no analysis mode
#     chunks={}, # allows to open the default chunking
# )

In [14]:
s2l2a_zarr_sample= xr.open_datatree(
    url, engine="zarr")

/var/folders/hr/vlzgj7wj51l7ps0dxp4wp08r0000gn/T/ipykernel_2746/1054942646.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  s2l2a_zarr_sample= xr.open_datatree(
/var/folders/hr/vlzgj7wj51l7ps0dxp4wp08r0000gn/T/ipykernel_2746/1054942646.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  s2l2a_zarr_sample= xr.open_datatree(


If we apply the helper function `print_gen_structure` on the root of the `DataTree` object, we will get a listing of the tree-like structure of the object. We can see all Zarr groups, such as `measurements`, `quality` and `conditions`, their sub-groups and content.

In [15]:
print("Zarr Sentinel 2 L2A Structure")
print_gen_structure(s2l2a_zarr_sample.root) 
print("-" * 30)

Zarr Sentinel 2 L2A Structure
None
  conditions
    geometry
    mask
      detector_footprint
        r10m
        r20m
        r60m
      l1c_classification
        r60m
      l2a_classification
        r20m
        r60m
    meteorology
      cams
      ecmwf
  measurements
    reflectance
      r10m
      r20m
      r60m
  quality
    atmosphere
      r10m
      r20m
      r60m
    l2a_quicklook
      r10m
      r20m
      r60m
    mask
      r10m
      r20m
      r60m
    probability
      r20m
------------------------------


## Extract information from Zarr groups

In the next step, we can explore the content of individually contained `.zarr` groups. By specifying the name of the group and subgroup and adding it into square brackets, we can extract the content of the relevant group. Let us for example, extract the content of the subgroup `reflectance` under `measurements`.

As a result, it is visible that there are three subgroups of the parent node `measurements/reflectance`: `r10`, `r20` and `r60`, which are the `DataArrays` with the three different resolutions of the Sentinel-2 L2A data.

The `xarray.DataTree` structure allows the exploration of additional group-related metadata and information. For example, we can find the `chunksize` of each array and the coordinates.

In [16]:
# # Retrieving the reflectance groups:
# s2l2a_zarr_sample["measurements/reflectance"] # Run it yourself for an inteactive overview

## Extract Zarr metadata on different levels

Through `s2l2a_zarr_sample.attrs[]` we are able to visualise both the `stac_discovery` and `other_metadata` included in the `zarr` store. <br>
<br>
For the properties inside `stac_discovery` for example we can get the parameters included:

In [17]:
# STAC metadata style:
print(list(s2l2a_zarr_sample.attrs["stac_discovery"].keys()))

['assets', 'bbox', 'geometry', 'id', 'links', 'properties', 'stac_extensions', 'stac_version', 'type']


We are also, able to retrieve specific information by diving deep into the `stac_discovery` metadata, such as:

In [18]:
print('Date of Item Creation: ', s2l2a_zarr_sample.attrs['stac_discovery']['properties']['created'])
print('Item Bounding Box    : ', s2l2a_zarr_sample.attrs['stac_discovery']['bbox'])
print('Item ESPG            : ', s2l2a_zarr_sample.attrs['stac_discovery']['properties']['proj:epsg'])
print('Sentinel Platform    : ', s2l2a_zarr_sample.attrs['stac_discovery']['properties']['platform'])
print('Item Processing Level: ', s2l2a_zarr_sample.attrs['stac_discovery']['properties']['processing:level'])

Date of Item Creation:  2025-06-10T13:20:01+00:00
Item Bounding Box    :  [9.146276872400831, 52.25344953517325, 7.500940412097549, 53.24953673463324]
Item ESPG            :  32632
Sentinel Platform    :  sentinel-2c
Item Processing Level:  L2A


And from `other_metadata`, we are able to retrieve the information specific to the instrument variables.

In [19]:
# Complementing metadata:
print(list(s2l2a_zarr_sample.attrs["other_metadata"].keys()))

['AOT_retrieval_model', 'L0_ancillary_data_quality', 'L0_ephemeris_data_quality', 'NUC_table_ID', 'SWIR_rearrangement_flag', 'UTM_zone_identification', 'absolute_location_assessment_from_AOCS', 'band_description', 'declared_accuracy_of_AOT_model', 'declared_accuracy_of_radiative_transfer_model', 'declared_accuracy_of_water_vapour_model', 'electronic_crosstalk_correction_flag', 'eopf_category', 'geometric_refinement', 'history', 'horizontal_CRS_code', 'horizontal_CRS_name', 'mean_sensing_time', 'mean_sun_azimuth_angle_in_deg_for_all_bands_all_detectors', 'mean_sun_zenith_angle_in_deg_for_all_bands_all_detectors', 'mean_value_of_aerosol_optical_thickness', 'mean_value_of_total_water_vapour_content', 'meteo', 'multispectral_registration_assessment', 'onboard_compression_flag', 'onboard_equalization_flag', 'optical_crosstalk_correction_flag', 'ozone_source', 'ozone_value', 'percentage_of_degraded_MSI_data', 'planimetric_stability_assessment_from_AOCS', 'product_quality_status', 'reflectanc

## 💪 Now it is your turn

As we are able to retrieve several items from the [EOPF Sentinel Zarr Samples Service STAC API](https://stac.browser.user.eopf.eodc.eu/), let us try the following:
<br>

### Task
Visit the [Sentinel-2 Level-2A collection](https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a) and:

- Choose an item of interest.
- Replicate the workflow and explore the item's metadata. When was it retrieved?
- What are the dimensions?
- What is the detailed location of the item?

## Conclusion
This tutorial provides an initial understanding of the `zarr` structure for a Sentinel-2 L2A product sample. By using the `xarray` library, we can effectively navigate and inspect the different components within the `zarr` format, including its metadata and array organisation. 

## What's next?

Now that you've been introduced to the `.zarr` encoding format, learned its core concepts, and understood the basics of how to explore it, you are prepared for the next step. In the following [chapter](./31_stac_intro.qmd) we will introduce you to **STAC** and the **EOPF Zarr STAC Catalog**. As we go along, we are more and more transition from theory to practice, providing you with hands-on tutorials working with EOPF `.zarr` products.